In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By 

driver = webdriver.Chrome()
url = "https://taas.koroad.or.kr/sta/acs/exs/typical.do?menuId=WEB_KMP_OVT_UAS_ASA"
driver.get(url)

In [43]:
# 좌측 교통사고 (경찰DB) 클릭 
driver.find_element(By.CSS_SELECTOR,"#jdMenuTree17").click()

In [44]:
# 사고일반
driver.find_element(By.CSS_SELECTOR,"#ddMenuTree17 > div:nth-child(5) > a.node").click()

In [45]:
# 사고유형별 시간대별 교통사고 클릭 
driver.find_element(By.CSS_SELECTOR,"#sdMenuTree32").click()

In [46]:
# 년도 2014년 시작
driver.find_element(By.CSS_SELECTOR,"#startYear > option:nth-child(10)").click() 

In [47]:
# 년도 2014년 끝
driver.find_element(By.CSS_SELECTOR,"#endYear > option:nth-child(10)").click()   

In [48]:
# 조회
driver.find_element(By.CSS_SELECTOR,"#searchDiv > ul.top01-03 > li:nth-child(2) > input").click()

In [49]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# iframe 요소 찾기
iframe = WebDriverWait(driver,10).until(
                                            EC.presence_of_element_located((By.TAG_NAME,"iframe"))
                                        )
# iframe 으로 전환 
driver.switch_to.frame(iframe)

In [50]:
# 전환된 iframe 내 html
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup

<html><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=EmulateIE7" http-equiv="X-UA-Compatible"/>
<title>Typical Analysis</title>
<link href="../common/img/yain_weblogo.gif" rel="shortcut icon"/>
<link href="/OBIP/common/css/common.css" rel="stylesheet" type="text/css"/>
<link href="/OBIP/analysis/css/analysis.css" rel="stylesheet" type="text/css"/>
<link href="/OBIP/analysis/octagon/css/octagon.css" rel="stylesheet" type="text/css"/>
<link href="/OBIP/analysis/octagon/css/style.css" rel="stylesheet" type="text/css"/>
<link href="/OBIP/OctagonGridStyleCss.jsp?styleId=5" id="styleInf" rel="stylesheet" type="text/css"/>
<script src="/OBIP/common/js/prototype.js" type="text/javascript"></script>
<script src="/OBIP/common/js/jquery-1.9.1.js" type="text/javascript"></script>
<script src="/OBIP/common/js/jquery-ui.js" type="text/javascript"></script>
<script type="text/javascript">
jQuery.noConflict();
var j$ = jQuery;
</script>
<!--script type="t

In [ ]:
# table 접근 후 데이터 파싱 
label_rows = soup.select("#OctagonGrid > tbody > tr")

for row in label_rows:
    cell = row.find_all("td")
    if len(cell) == 2:
        

<tr><td code="차대사람" coldepth="3" level="0" rows="15" rowspan="15"><nobr>차대사람</nobr></td><td code="횡단중" coldepth="2" level="0" rows="3" rowspan="3"><nobr>횡단중</nobr></td><td &nbsp;type="16" class="rowMeaTitle" coldepth="1" level="0" meaidx="0"><nobr>사고[건]</nobr></td></tr>
<tr><td &nbsp;type="16" class="rowMeaTitle" coldepth="1" level="0" meaidx="1"><nobr>사망[명]</nobr></td></tr>
<tr><td &nbsp;type="16" class="rowMeaTitle" coldepth="1" level="0" meaidx="2"><nobr>부상[명]</nobr></td></tr>
<tr><td code="차도통행중" coldepth="2" level="0" rows="3" rowspan="3"><nobr>차도통행중</nobr></td><td &nbsp;type="16" class="rowMeaTitle" coldepth="1" level="0" meaidx="0"><nobr>사고[건]</nobr></td></tr>
<tr><td &nbsp;type="16" class="rowMeaTitle" coldepth="1" level="0" meaidx="1"><nobr>사망[명]</nobr></td></tr>
<tr><td &nbsp;type="16" class="rowMeaTitle" coldepth="1" level="0" meaidx="2"><nobr>부상[명]</nobr></td></tr>
<tr><td code="길가장자리구역통행중" coldepth="2" level="0" rows="3" rowspan="3"><nobr>길가장자리구역통행중</nobr></td><td &nbsp;ty

In [27]:
# 표 로딩 대기 
table = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR,"#OctagonPanel_1 > div.ovReportArea > div.ovReportArea.ovGridPanel"))
)

# 모든 행 가져오기
rows = table.find_elements(By.TAG_NAME,"tr")

for row in rows:
    print(row.text)

사고유형대분류 사고유형중분류 구분

00시-02시 02시-04시 04시-06시
차대사람 횡단중 사고[건]
사망[명]
부상[명]
차도통행중 사고[건]
사망[명]
부상[명]
길가장자리구역통행중 사고[건]
사망[명]
부상[명]
사고[건]





























950 484 652 1,080 1,490 1,192 1,350 1,679 1,993 2,707 2,241 1,726
61 36 103 103 34 50 34 36 55 148 124 99
991 525 601 1,014 1,521 1,206 1,379 1,710 2,049 2,742 2,304 1,796
183 105 115 123 239 227 237 285 340 458 431 316
16 11 13 8 4 7 5 8 13 20 19 22
183 97 109 120 242 224 237 286 338 463 451 311
149 92 91 121 274 206 259 294 365 362 334 325
4 2 6 3 6 8 6 7 8 12 13 5
152 99 93 122 276 220 260 296 379 367 344 342
77 40 44 94 180 213 231 242 280 314 285 188
0 1 6 2 4 1 6 3 5 11 5 1
87 45 42 98 185 224 236 261 307 343 323 214
1,403 805 658 1,130 2,069 1,955 2,222 2,589 3,060 3,551 2,823 2,387
45 51 49 60 49 50 43 46 51 107 73 65
1,482 820 656 1,124 2,094 1,984 2,273 2,684 3,175 3,660 2,926 2,498
3,331 2,036 2,010 4,224 6,331 5,756 5,950 6,502 7,220 7,766 6,294 5,355
39 28 32 93 69 89 73 71 110 95 61 54
6,072 3,750 3,384 6,915 10,052

In [31]:
# 데이터 추출 
data = []
for row in rows:
    cells = row.find_elements(By.TAG_NAME,"td")
    row_data = [cell.text for cell in cells]
    if row_data:
        data.append(row_data)
    print()
# driver.quit()

In [16]:
import pandas as pd 
df = pd.DataFrame(data)
df.to_csv(r"C:\Users\Playdata\Documents\MyRepo\04_web_crawling\사고유형별_시간대별_교통사고2.csv",index=False)